In [ ]:
# !pip install tensorflow librosa matplotlib joblib glob

In [1]:
import numpy as np
import os
import librosa
import joblib
from glob import glob
import IPython.display as ipd

In [2]:
example_audio = glob('/home/gta/chiacchio/ProcVoz/dataEmotion/*/*.wav')[0]

In [3]:
# Resample the audio to a different sample rate
resampled_audio, _ = librosa.load(example_audio, sr=4000)
ipd.Audio(resampled_audio, rate=4000)
# ipd.Audio(example_audio)

Ler os arquivos de áudio para coletar os coeficientes e os os labels. Os labels serão divididos da seguinte maneira:
- Anger: 0
- Disgust: 1
- Fear: 2
- Happy: 3
- Neutral: 4
- Sad: 5

In [4]:
sr = 4000
x, sr = librosa.load(example_audio, sr=sr)
x = librosa.util.fix_length(x, size=sr*10)
print(x, sr, len(x))

librosa.feature.mfcc(y=x, sr=sr, n_mfcc=64).shape

[0.00223407 0.00304511 0.00159415 ... 0.         0.         0.        ] 4000 40000


(64, 79)

In [5]:
base_dir = '/home/gta/chiacchio/ProcVoz/dataEmotion/'

In [6]:
len(glob(base_dir + '*/*.wav'))

54490

In [7]:
max_len = max([librosa.get_duration(path=audio_file) for audio_file in glob('dataEmotion/*/*.wav')])
min_len = min([librosa.get_duration(path=audio_file) for audio_file in glob('dataEmotion/*/*.wav')])
print('min_index: ', [librosa.get_duration(path=audio_file) for audio_file in glob('dataEmotion/*/*.wav')].index(min_len))
min_len, max_len

ValueError: max() arg is an empty sequence

In [11]:
label_map = {'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5}
lst = []
number_of_audios_per_class = -1 # set -1 for all audios
sr = 4000
n_mfcc = 64

for emotion in label_map.keys():
    folder_path = os.path.join(base_dir, emotion)
    audios_number = 0
    # Iterate over the audio files in the folder
    for audio_file in glob(base_dir + emotion + '/*.wav'):
        # print(os.path.join(folder_path, audio_file))
        x, sample_rate = librosa.load(audio_file, sr=sr)
        x = librosa.util.fix_length(x, size=int(sr*10))
        mfccs = np.array(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=n_mfcc))
        label = label_map[emotion]
        arr = mfccs, label
        lst.append(arr)
        audios_number += 1
        if(audios_number == number_of_audios_per_class): break
        print(f'\r{audios_number} {emotion} audio files loaded', end='')
    print(audios_number, emotion, 'audio files loaded')

9315 anger audio files loaded9315 anger audio files loaded
9315 disgust audio files loaded9315 disgust audio files loaded
9315 fear audio files loaded9315 fear audio files loaded
9315 happy audio files loaded9315 happy audio files loaded
7915 neutral audio files loaded7915 neutral audio files loaded
9315 sad audio files loaded9315 sad audio files loaded


In [12]:
x, y = zip(*lst)

In [13]:
x = np.asarray(x)
y = np.asarray(y)

In [14]:
import joblib

In [15]:


X_name = 'x.joblib'
y_name = 'y.joblib'
save_dir = base_dir

savedX = joblib.dump(x, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [7]:
x = joblib.load(base_dir + 'x.joblib')
y = joblib.load(base_dir + 'y.joblib')

In [8]:
input_shape = x[0].shape
input_shape, x.shape, y.shape

((64, 79), (54490, 64, 79), (54490,))

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

## Neural Network

In [10]:
# x_traincnn = np.expand_dims(X_train, axis=2)
# x_testcnn = np.expand_dims(X_test, axis=2)

x_traincnn = X_train
x_testcnn = X_test

In [11]:
x_traincnn.shape, x_testcnn.shape

((36508, 64, 79), (17982, 64, 79))

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [13]:
import keras
import numpy as np
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# List available GPUs
print("Available GPUs: ", tf.config.experimental.list_physical_devices('GPU'))

2024-11-13 21:37:31.709354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731544651.722803  539126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731544651.726613  539126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 21:37:31.739995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  0
Available GPUs:  []


2024-11-13 21:37:32.776322: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-13 21:37:32.776364: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: paodeacucar
2024-11-13 21:37:32.776369: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: paodeacucar
2024-11-13 21:37:32.776435: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 565.57.1
2024-11-13 21:37:32.776450: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 565.57.1
2024-11-13 21:37:32.776454: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 565.57.1


In [48]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.45      0.84      0.59      3167
           1       0.53      0.48      0.50      3065
           2       0.69      0.38      0.49      3002
           3       0.60      0.45      0.51      3114
           4       0.65      0.50      0.56      2568
           5       0.56      0.62      0.59      3066

    accuracy                           0.55     17982
   macro avg       0.58      0.54      0.54     17982
weighted avg       0.58      0.55      0.54     17982



## Modelo https://www.kaggle.com/code/shivamburnwal/speech-emotion-recognition?scriptVersionId=34958802&cellId=52

Acurácia de 0.79

In [14]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [15]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(units=6, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 64, 256)        │       101,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 32, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 256)        │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 16, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 16, 128)        │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 8, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 8, 64)          │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,878 (2.46 MB)

 Trainable params: 643,302 (2.45 MB)

 Non-trainable params: 576 (2.25 KB)

In [16]:
from tensorflow.keras.utils import to_categorical

In [17]:
y_train_encoded = to_categorical(y_train, num_classes=6)
y_test_encoded = to_categorical(y_test, num_classes=6)

In [18]:
y_train_encoded.shape

(36508, 6)

In [19]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience=3)
history=model.fit(x_traincnn, y_train_encoded, batch_size=64, epochs=100, validation_data=(x_testcnn, y_test_encoded), callbacks=[callback])

Epoch 1/100


2024-11-13 21:37:47.631424: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 738337792 exceeds 10% of free system memory.


569/571 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2859 - loss: 1.7668

2024-11-13 21:38:05.275911: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363667968 exceeds 10% of free system memory.


571/571 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.2864 - loss: 1.7655 - val_accuracy: 0.5482 - val_loss: 1.1757
Epoch 2/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 17s 30ms/step - accuracy: 0.5451 - loss: 1.1781 - val_accuracy: 0.6098 - val_loss: 1.0091
Epoch 3/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 18s 31ms/step - accuracy: 0.5849 - loss: 1.0788 - val_accuracy: 0.5710 - val_loss: 1.1489
Epoch 4/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 18s 31ms/step - accuracy: 0.6327 - loss: 0.9781 - val_accuracy: 0.6499 - val_loss: 0.9116
Epoch 5/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 18s 31ms/step - accuracy: 0.6752 - loss: 0.8884 - val_accuracy: 0.6920 - val_loss: 0.8129
Epoch 6/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.7025 - loss: 0.8138 - val_accuracy: 0.6984 - val_loss: 0.8158
Epoch 7/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.7346 - loss: 0.7357 - val_accuracy: 0.6596 - val_loss: 0.9345
Epoch 8/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.7614 - loss: 0.6677 - val_

In [20]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

 30/562 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

2024-11-13 21:57:09.701817: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363667968 exceeds 10% of free system memory.


562/562 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [21]:
predictions_encoded = to_categorical(predictions, num_classes=6)

In [22]:
from sklearn.metrics import classification_report
report = classification_report(y_test_encoded, predictions_encoded)
print(report)

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      3167
           1       0.80      0.85      0.82      3065
           2       0.86      0.81      0.83      3002
           3       0.90      0.80      0.84      3114
           4       0.86      0.86      0.86      2568
           5       0.79      0.86      0.82      3066

   micro avg       0.85      0.85      0.85     17982
   macro avg       0.85      0.85      0.85     17982
weighted avg       0.85      0.85      0.85     17982
 samples avg       0.85      0.85      0.85     17982



              precision    recall  f1-score   support

           0       0.87      0.94      0.90       863
           1       0.75      0.58      0.65       290

   micro avg       0.85      0.85      0.85      1153
   macro avg       0.81      0.76      0.78      1153
weighted avg       0.84      0.85      0.84      1153
 samples avg       0.85      0.85      0.85      1153

In [23]:
MODEL_DIR_PATH = 'models'

In [24]:
if not os.path.isdir(MODEL_DIR_PATH):
    os.makedirs(MODEL_DIR_PATH)
model_path = os.path.join(MODEL_DIR_PATH, 'emotion_detection_2.keras')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at models/emotion_detection_2.keras 
